
### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2020 Semester 1

## Assignment 1: Naive Bayes Classifiers

###### Submission deadline: 7 pm, Monday 20 Apr 2020

**Student Name(s):**    `PLEASE ENTER YOUR NAME(S) HERE`

**Student ID(s):**     `PLEASE ENTER YOUR ID(S) HERE`


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [1]:
import pandas as pd
import numpy as np
import math
from scipy import stats
from copy import copy, deepcopy

In [2]:
#    read all datasets 
adult = pd.read_csv("datasets/adult.data", header = None)
bank = pd.read_csv("datasets/bank.data", header = None)
breast_cancer_wisconsin = pd.read_csv("datasets/breast-cancer-wisconsin.data", header = None)
car = pd.read_csv("datasets/car.data", header = None)
lymphography = pd.read_csv("datasets/lymphography.data", header = None)
mushroom = pd.read_csv("datasets/mushroom.data", header = None)
nursery = pd.read_csv("datasets/nursery.data", header = None)
somerville = pd.read_csv("datasets/somerville.data", header = None)
university = pd.read_csv("datasets/university.data", header = None)
wdbc = pd.read_csv("datasets/wdbc.data", header = None)
wine = pd.read_csv("datasets/wine.data", header = None)

In [3]:
#   Helper function that delete all missing values
def delete_missing_value(raw_dataset, missing_values):
    rows = set(raw_dataset[raw_dataset.values == missing_values].index)
    data = raw_dataset.drop(index = rows)
    return data

In [4]:

#    This function should prepare the data by reading it from a file and
#    converting it into a useful format for training and testing

def preprocess(df, missing_values):
#     train=df.sample(frac=0.8,random_state=200)
#     test=df.drop(train.index)
#     Delete all missing values
    train = delete_missing_value(df, missing_values)
    return train


In [24]:
# Global Variable

#  Enter dataset here
missing_values = '?'
train_data = preprocess(mushroom, missing_values)

'''
    datasets_types can be 
    NOMINAL: NOMINAL ATTRIBUTES DATASETS, 
    NUMERIC: NUMERIC ATTRIBUTES DATASETS, 
    ORDINAL: ORDINAL ATTRIBUTES DATASETS, 
    MIX: DATASETS WITH A MIX OF ATTRIBUTE TYPES
'''
datasets_types = "NOMINAL"

# 0 represents nomianl, 1 represents ordinal, 2 represents numeric
feature_types = [2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 0]
class_column = 0
EPSILON = 1/(2*train_data.shape[0])

# 分开label和数据
label = train_data.iloc[:,class_column]
train_set = train_data.drop(columns=class_column)

feature_columns = list(train_set.columns)

# Show some useful information about the data
print("Data Shape =====>", train_data.shape)
print("======================================================")
print("EPSILON ========>", EPSILON)
print("======================================================")
print("Some Data Examples:")
display(train_data.head(3))
print("======================================================")
print("Data Type:")
display(train_data.dtypes)

Data Shape =====> (5644, 23)
EPSILON ========> 8.858965272856131e-05
Some Data Examples:


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m


Data Type:


0     object
1     object
2     object
3     object
4     object
5     object
6     object
7     object
8     object
9     object
10    object
11    object
12    object
13    object
14    object
15    object
16    object
17    object
18    object
19    object
20    object
21    object
22    object
dtype: object

In [6]:
#calculate prior
def find_nominal_prob(class_column):
    prior = {}
    total_number = len(class_column)
    classes = class_column.value_counts()
    for key in classes.keys():
        prior[key] = math.log2(classes[key]/total_number)
    return prior

#Tutor版本:
def nominal_prob(series,denom,n_keys,k=0): 
    prior = {}
    counts = series.value_counts()
    keys = list(counts.keys())
    for key in keys: 
        prior[key] = np.log2((counts[key]+k)/(denom+k*n_keys)) 
    return prior 


In [7]:
# Calculate prior
print(find_nominal_prob(adult[class_column]))

{36: -5.18028575254781, 31: -5.196441520938851, 34: -5.1996944987339395, 23: -5.214424354844224, 35: -5.216070327687784, 33: -5.217718180569267, 28: -5.230969204067122, 30: -5.240987959892114, 37: -5.246023549789411, 25: -5.274895397033813, 27: -5.285224999927543, 32: -5.297370429789632, 38: -5.299113868117683, 39: -5.318432045317462, 29: -5.32374584521393, 41: -5.332645904537163, 24: -5.3506124510666115, 40: -5.357862190145999, 26: -5.374308543509968, 42: -5.383527073539347, 43: -5.402142751706694, 22: -5.411541449708944, 20: -5.434351332617029, 46: -5.465336578193888, 45: -5.471221134445745, 44: -5.491011500205753, 21: -5.499004290959283, 19: -5.51512395632256, 47: -5.52325183720596, 50: -5.757237710529256, 51: -5.774111529093652, 49: -5.818429878643475, 18: -5.887569578876936, 48: -5.906048999484596, 52: -6.089990579308209, 53: -6.132876392161386, 55: -6.280050953589606, 54: -6.29388986105467, 17: -6.365148544224493, 58: -6.475157549004915, 56: -6.475157549004915, 57: -6.50704161002

In [8]:
def find_nominal_llh(attributes, llh_dict, data, feature_row):
    all_feature = data[feature_row].value_counts()
    for attribute in attributes:
        feature_dict = {feature_row:{}}
        subset = data[data[class_column] == attribute][feature_row]
        feature_dict[feature_row] = find_nominal_prob(subset)
        for check_feature in all_feature.keys():
            if check_feature not in feature_dict[feature_row].keys():
                feature_dict[feature_row][check_feature] = np.log2(EPSILON)
        if(llh_dict[attribute]):
            llh_dict[attribute].update(feature_dict)
        else:
            llh_dict[attribute] = feature_dict
            
    return llh_dict

In [9]:
def find_numerical_llh(attributes, llh_dict, data, feature):
#  Save mean and Std as llh
    for attribute in attributes:
        feature_dict = {feature:{}}
        subset = subset = data[data[class_column] == attribute][feature]
        mean = np.mean(subset)
        std = np.std(subset)
        feature_dict[feature]["mean"] = mean
        feature_dict[feature]["std"] = std
        if(llh_dict[attribute]):
            llh_dict[attribute].update(feature_dict)
        else:
            llh_dict[attribute] = feature_dict
    return llh_dict

In [16]:
#  This function should calculat prior probabilities and likelihoods from the training data
#  and using them to build a naive Bayes model

def train(data):
    prior = find_nominal_prob(data[class_column])
    llh_dict = dict.fromkeys(prior)
    for i in feature_columns:
        if(datasets_types == "NUMERIC" or (datasets_types == "MIX" and feature_types[i] == 2)):
            find_numerical_llh(prior, llh_dict, data, i)
        else:
            find_nominal_llh(prior, llh_dict, data, i)
            
    return prior, llh_dict

prior, llh_dict = train(train_data)


In [17]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set)

def predict(instance, prior, llh_dict):
    all_prob = {}
    for attribute in prior.keys():
        all_prob[attribute] = prior[attribute]
        for i in feature_columns:
            prob = 0
            if(datasets_types == "NUMERIC" or (datasets_types == "MIX" and feature_types[i] == 2)):
                mean = max(llh_dict[attribute][i]["mean"], 1 * 10 ** -8)
                std = max(llh_dict[attribute][i]["std"], 1 * 10 ** -8)
                prob = stats.norm.pdf(x=instance[i], loc=mean, scale=std)
                if prob > 0.0:
                    prob = np.log2(prob)
                else:
                    prob = np.log2(1 * 10 ** -8)
            else:
                prob = llh_dict[attribute][i][instance[i]]
            all_prob[attribute] += prob
    max_prob = -10000
    max_key = ""
    
    for prob in all_prob.keys():
        if(all_prob[prob] > max_prob):
            max_prob = all_prob[prob]
            max_key = prob
    return max_key

In [18]:
predict(train_data.iloc[0], prior, llh_dict)

'p'

In [19]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate(data, prior, llh_dict, interesting_class = None):
    predict_list = []
    correct = 0
    total = data.shape[0]
    result = {}
    TP = 0
    FN = 0
    FP = 0
    TN = 0
    
    for index, row in data.iterrows():
        actual_class = row[class_column]
        predict_class = predict(row, prior, llh_dict)
        if (actual_class == predict_class):
            correct += 1
            if interesting_class:
                if(interesting_class == row[class_column]):
                    TP += 1
                elif(interesting_class != row[class_column]):
                    TN += 1
        else:
            if interesting_class:
                if(interesting_class == row[class_column]):
                    FP += 1
                elif(interesting_class != row[class_column]):
                    FN += 1
                    
    correct_rate = correct/total
    result["correct_rate"] = correct_rate
    
    if (interesting_class):
        precision = TP/(TP + FP)
        recall = TP/(TP + FN)
        result["precision"] = precision
        result["recall"] = recall
    print("correct",correct,total)
    return result
# %time
print("======================================================")
print("Overall Accuracy is: ")
print(evaluate(train_data, prior, llh_dict))

Overall Accuracy is: 
correct 5592 5644
{'correct_rate': 0.9907866761162296}


## Questions 


If you are in a group of 1, you will respond to question (1), and **one** other of your choosing (two responses in total).

If you are in a group of 2, you will respond to question (1) and question (2), and **two** others of your choosing (four responses in total). 

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions in respond to the question, but your formal answer should be added to a separate file.

### Q1
Try discretising the numeric attributes in these datasets and treating them as discrete variables in the na¨ıve Bayes classifier. You can use a discretisation method of your choice and group the numeric values into any number of levels (but around 3 to 5 levels would probably be a good starting point). Does discretizing the variables improve classification performance, compared to the Gaussian na¨ıve Bayes approach? Why or why not?

In [25]:
#  Use adult dataset for this question
missing_values = '?'
train_data = preprocess(adult, missing_values)

'''
    datasets_types can be 
    NOMINAL: NOMINAL ATTRIBUTES DATASETS, 
    NUMERIC: NUMERIC ATTRIBUTES DATASETS, 
    ORDINAL: ORDINAL ATTRIBUTES DATASETS, 
    MIX: DATASETS WITH A MIX OF ATTRIBUTE TYPES
'''
datasets_types = "MIX"

# 0 represents nomianl, 1 represents ordinal, 2 represents numeric
feature_types = [2, 0, 2, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 0]
class_column = 14
EPSILON = 1/(2*train_data.shape[0])

# 分开label和数据
label = train_data.iloc[:,class_column]
train_set = train_data.drop(columns=class_column)

feature_columns = list(train_set.columns)


def discretise(data,data_type,levels,by):
    #给定一个数据集及其对应的数据类型，
    #必须将其所有数字特征离散为离散特征，
    #'宽度' & '频率'
    #代表等宽或者等频率并且取一个可靠的整数，
    #然后确定离散后的特征级别的总数
    copy_data = data.copy()
    discrete_datatype = data_type.copy()
    for i in range(len(data_type)):
        # 找出所有的 Numeric Data,然后组成新的DataFrame
        # Note: 2 means numeric data type
        if data_type[i] == 2: # continuous
            feature = data.iloc[:,i]
            copy_feature = feature.copy()
            
            # Equal Width
            if by=='width':
                # calculate width of each level
                maximum = np.max(feature)
                minimum = np.min(feature)
                width = (maximum-minimum)/levels # width of each level
                for j in range(levels):
                    copy_feature[feature<=(maximum-width*j)] = ('level'+str(levels-j))
           
            # Equal Quantile
            else:
                frequency = feature.shape[0]/levels # frequency of each level
                for m in range(levels):
                    lower = int(frequency*m) # lower bound of index, so has to be integer
                    upper = int(frequency*(m+1)) # upper bound of index, so has to be integer
                    copy_feature[feature.sort_values()[lower:upper].index] = ('level'+str(levels-m))
            copy_data.iloc[:,i] = copy_feature
            discrete_datatype[i] = 1 
    return copy_data,discrete_datatype

In [26]:
width_data,discrete_datatype = discretise(train_data,feature_types,levels=5,by='width')
freq_data,discrete_datatype = discretise(train_data,feature_types,levels=5,by='freq')
print(discrete_datatype)
print("======================================================")
print("Equal Width:")
display(width_data.iloc[:,10].value_counts())
print("======================================================")
print("Equal Frequency:")
display(freq_data.iloc[:,10].value_counts())

[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0]
Equal Width:


level1    29930
level5      148
level2       82
level3        2
Name: 10, dtype: int64

Equal Frequency:


level1    6033
level3    6033
level4    6032
level2    6032
level5    6032
Name: 10, dtype: int64

In [27]:
display(width_data.head(5))
width_data[2].value_counts()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,level2,State-gov,level1,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,level1,level1,level2,United-States,<=50K
1,level3,Self-emp-not-inc,level1,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,level1,level1,level1,United-States,<=50K
2,level2,Private,level1,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,level1,level1,level2,United-States,<=50K
3,level3,Private,level1,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,level1,level1,level2,United-States,<=50K
4,level1,Private,level2,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,level1,level1,level2,Cuba,<=50K


level1    26380
level2     3652
level3      111
level4       14
level5        5
Name: 2, dtype: int64

In [28]:
feature_types = discrete_datatype
width_prior, width_llh_dict = train(width_data)
width_eval = evaluate(width_data, width_prior, width_llh_dict)
print(width_eval)

correct 24524 30162
{'correct_rate': 0.8130760559644586}


In [33]:
freq_prior, freq_llh_dict = train(freq_data)
freq_eval = evaluate(freq_data, freq_prior, freq_llh_dict)
print(freq_eval)

correct 24439 30162
{'correct_rate': 0.810257940454877}


### Q2
Implement a baseline model (e.g., random or 0R) and compare the performance of the na¨ıve Bayes classifier to this baseline on multiple datasets. Discuss why the baseline performance varies across datasets, and to what extent the na¨ıve Bayes classifier improves on the baseline performance.

In [34]:
def zero_r_evaluate(data,label_index):
    temp_data = data.copy()
    zero_r_result = temp_data[label_index].value_counts().idxmax()

    series_of_class = temp_data[label_index]
    list_of_class = series_of_class.values.tolist()
    
    correct = 0
    total = data.shape[0]

    for i in range(len(list_of_class)):
        if zero_r_result == list_of_class[i]:
            correct += 1
    correct_rate = correct / total
    
    return correct_rate

In [35]:
print("Some 0R baseline model for varius dataset:")

print()

print("0R baseline model performance for ADULT dataset:")
adult_0R_data = preprocess(adult,"?") 
print(zero_r_evaluate(adult_0R_data,14))
print("======================================================")

print("0R baseline model performance for MUSHROOM dataset:")
mushroom_0R_data = preprocess(mushroom,"?") 
print(zero_r_evaluate(mushroom_0R_data,14))
print("======================================================")

print("0R baseline model performance for BANK dataset:")
bank_0R_data = preprocess(bank,None) 
print(zero_r_evaluate(bank_0R_data,14))
print("======================================================")

print("0R baseline model performance for WINE dataset:")
wine_0R_data = preprocess(wine,None) 
print(zero_r_evaluate(wine_0R_data,0))
print("======================================================")

Some 0R baseline model for varius dataset:

0R baseline model performance for ADULT dataset:
0.7510775147536636
0R baseline model performance for MUSHROOM dataset:
0.5556343019135365
0R baseline model performance for BANK dataset:
0.8830151954170445
0R baseline model performance for WINE dataset:
0.398876404494382


### Q3
Since it’s difficult to model the probabilities of ordinal data, ordinal attributes are often treated as either nominal variables or numeric variables. Compare these strategies on the ordinal datasets provided. Deterimine which approach gives higher classification accuracy and discuss why.

In [36]:
# use car dataset for this question
car.head(10)

,0,1,2,3,4,5,6
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
5,vhigh,vhigh,2,2,med,high,unacc
6,vhigh,vhigh,2,2,big,low,unacc
7,vhigh,vhigh,2,2,big,med,unacc
8,vhigh,vhigh,2,2,big,high,unacc
9,vhigh,vhigh,2,4,small,low,unacc


In [37]:
# https://stackoverflow.com/questions/21818886/changing-ordinal-character-data-to-numeric-data-with-pandas
car_origin = pd.read_csv("datasets/car.data", header = None)
car = car_origin.copy()
buying_conv_dict= {'low':0,'med':1,'high':2,'vhigh':3}
car[0]=car[0].apply(buying_conv_dict.get)
maint_conv_dict = {'low':0,'med':1,'high':2,'vhigh':3}
car[1]=car[1].apply(maint_conv_dict.get)
doors_conv_dict = {'2':1, '3':2, '4':3, '5more': 4}
car[2]=car[2].apply(doors_conv_dict.get)
persons_conv_dict = {'2':0, '4':1, 'more':2}
car[3]=car[3].apply(persons_conv_dict.get)
lug_boot_conv_dict = {'small':0,'med':1, 'big':2}
car[4]=car[4].apply(lug_boot_conv_dict.get)
safety_conv_dict = {'low':0 , 'med':1, 'high':2}
car[5]=car[5].apply(safety_conv_dict.get)
class_conv_dict = {'unacc':0 , 'acc':1, 'good':2, 'vgood':3}
car[6]=car[6].apply(class_conv_dict.get)

#nominal 
datasets_types = "NOMINAL"
class_column = 6
label = car_origin.iloc[:,class_column]
train_set = car_origin.drop(columns=class_column)
feature_columns = list(train_set.columns)
prior, llh_dict = train(car_origin)
print(evaluate(car_origin, prior, llh_dict))

#numercial 
datasets_types = "NUMERIC"
class_column = 6
prior, llh_dict = train(car)
print(evaluate(car, prior, llh_dict))


correct 1511 1729
{'correct_rate': 0.8739155581260845}
correct 1098 1729
{'correct_rate': 0.6350491613649508}


In [38]:
nursery.head(10)

,0,1,2,3,4,5,6,7,8
0,usual,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,usual,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,usual,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,usual,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,usual,proper,complete,1,convenient,convenient,slightly_prob,priority,priority
5,usual,proper,complete,1,convenient,convenient,slightly_prob,not_recom,not_recom
6,usual,proper,complete,1,convenient,convenient,problematic,recommended,priority
7,usual,proper,complete,1,convenient,convenient,problematic,priority,priority
8,usual,proper,complete,1,convenient,convenient,problematic,not_recom,not_recom
9,usual,proper,complete,1,convenient,inconv,nonprob,recommended,very_recom


In [39]:
nursery_origin = pd.read_csv("datasets/nursery.data", header = None)

nursery_numeric = nursery_origin.copy()

parents_dict = {'usual':0, 'pretentious':1, 'great_pret':2}
nursery_numeric[0]=nursery_numeric[0].apply(parents_dict.get)
has_nurs_dict ={'proper':0, 'less_proper':1, 'improper':2, 'critical':3, 'very_crit':4}
nursery_numeric[1]=nursery_numeric[1].apply(has_nurs_dict.get)
form_dict = {'complete':0, 'completed':1, 'incomplete':2, 'foster':3}
nursery_numeric[2]=nursery_numeric[2].apply(form_dict.get)
children_dict ={'1':0, '2':1, '3':2, 'more':3}
nursery_numeric[3]=nursery_numeric[3].apply(children_dict.get)
housing_dict = {'convenient':0, 'less_conv':1, 'critical':2}
nursery_numeric[4]=nursery_numeric[4].apply(housing_dict.get)
finance_dict = {'convenient':0, 'inconv':1}
nursery_numeric[5]=nursery_numeric[5].apply(finance_dict.get)
social_dict = {'nonprob':0, 'slightly_prob':1, 'problematic':2}
nursery_numeric[6]=nursery_numeric[6].apply(social_dict.get)
health_dict = {'recommended':0, 'priority':1, 'not_recom':2}
nursery_numeric[7]=nursery_numeric[7].apply(health_dict.get)
class_dict = {'not_recom':0, 'recommend':1, 'very_recom':2, 'priority':3, 'spec_prior':4}
nursery_numeric[8]=nursery_numeric[8].apply(class_dict.get)

#nominal 
datasets_types = "NOMINAL"
class_column = 8
label = nursery_origin.iloc[:,class_column]
train_set = nursery_origin.drop(columns=class_column)
feature_columns = list(train_set.columns)
prior, llh_dict = train(nursery_origin)
print(evaluate(nursery_origin, prior, llh_dict))

#numercial 
datasets_types = "NUMERIC"
class_column = 8
prior, llh_dict = train(nursery_numeric)
print(evaluate(nursery_numeric, prior, llh_dict))

correct 11704 12960
{'correct_rate': 0.9030864197530865}
correct 8201 12960
{'correct_rate': 0.6327932098765432}


### Q4
Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy (you should implement this yourself and do not simply call existing implementations from `scikit-learn`). How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

### Q5
Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the na¨ıve Bayes classifier? Explain why, or why not.

### Q6
The Gaussian na¨ıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in these datasets? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the NB classifier’s predictions.